In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!wget https://archive.ics.y
uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
!unzip smsspamcollection.zip


--2023-01-15 09:26:36--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203415 (199K) [application/x-httpd-php]
Saving to: ‘smsspamcollection.zip.3’

smsspamcollection.z 100%[===================>] 198.65K   946KB/s    in 0.2s    

2023-01-15 09:26:36 (946 KB/s) - ‘smsspamcollection.zip.3’ saved [203415/203415]

Archive:  smsspamcollection.zip
replace SMSSpamCollection? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: SMSSpamCollection       
replace readme? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: readme                  


In [11]:
import collections

def tokenize(s):
    return [t.rstrip('.') for t in s.split(' ')]

def vectorize(tokens):
    return collections.Counter(tokens)

def readiter(fi):
    for line in fi:
        fields = line.strip('\n').split('\t')
        x = vectorize(tokenize(fields[1]))
        y = fields[0]
        yield x, y

with open('SMSSpamCollection') as fi:
    D = [d for d in readiter(fi)]

In [14]:
from sklearn.model_selection import train_test_split

Dtrain, Dtest = train_test_split(D, test_size=0.1, random_state=0)

In [15]:
len(Dtrain), len(Dtest)

(5016, 558)

In [33]:
# Q1
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import DictVectorizer

VX = DictVectorizer()
VY = LabelEncoder()

Xtrain = VX.fit_transform([d[0] for d in Dtrain])
Ytrain = VY.fit_transform([d[1] for d in Dtrain])
Xtest = VX.transform([d[0] for d in Dtest])
Ytest = VY.transform([d[1] for d in Dtest])

In [114]:
import numpy as np
from scipy import sparse

w = sparse.csr_matrix(np.zeros((int(Xtrain.shape[1]), 1)))

max_epochs = 100000
eta0 = 0.5
alpha = 1e-6
Xtrain.shape[0]

def sigmoid(a):
    if 0 <= a:
        return 1 / (1 + np.exp(-a))
    else:
        return 1. - 1 / (1 + np.exp(a))

for t in range(max_epochs):
  eta = eta0 / np.sqrt(1+t)
  i = np.random.randint(0, Xtrain.shape[0])
  p = sigmoid(np.dot(Xtrain[i], w).toarray()[0])
  grad = (eta * (Ytrain[i] - p))[0]
  w += grad * sparse.csr_matrix.transpose(Xtrain[0])

In [116]:
# Q2
N = Xtest.shape[0]
OKCOUNT = 0
for i in range(N):
  y_hat = 1 if sigmoid(np.dot(Xtest[i], w).toarray()[0]) > 0.5 else 0
  if Ytest[i] == y_hat:
    OKCOUNT += 1
print(OKCOUNT/N)

0.8405017921146953


In [151]:
# Q3
F = sorted(zip(VX.feature_names_, [ws[0] for ws in w.toarray()]), key=lambda x: abs(x[1]), reverse=True)

print("Negative Weight:", [f for f in F[:20] if f[1] < 0])

print("Positive Weight:", [f for f in F[:20] if f[1] > 0])

Negative Weight: [('ME', -152.03850102706218), ('SO', -152.03850102706218), ('U', -152.03850102706218), ('9', -76.01925051353109), ('ABOUT', -76.01925051353109), ('AT', -76.01925051353109), ('BABE', -76.01925051353109), ('BE', -76.01925051353109), ('BED', -76.01925051353109), ('BUT', -76.01925051353109), ('CALL', -76.01925051353109), ('HI', -76.01925051353109), ('I', -76.01925051353109), ('IM', -76.01925051353109), ('IN', -76.01925051353109), ('LIKELY', -76.01925051353109), ('LOVE', -76.01925051353109), ('MOST', -76.01925051353109), ('R', -76.01925051353109), ('REALLY', -76.01925051353109)]
Positive Weight: []
